In [2]:
## Python code used to produce the results and figures for the paper:
## NCYPred: A Bidirectional Long Short-term Memory Network with Attention Mechanism 
##         for Y RNA and small non-coding RNA classification.

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import sys
import pickle
import tensorflow as tf
from tensorflow import keras
from Bio import SeqIO

sys.path.insert(1, './python-functions/')

from my_functions import CreateDf, SeqTo3mer, Remover, TokenPad

## Data processing and classification

In [ ]:
# load dataset
val_set = './dataset/validation-set-rfam.csv'
df_val = pd.read_csv(val_set, index_col=0)

# label one-hot encoding
one_hot_labels = pd.get_dummies(df_val['labels']).values
df_val['b-labels'] = list(one_hot_labels)

# formatting
array_list = []
for arr in list(df_val['b-labels']):
    array_list.append(arr)
    
y_b = np.vstack(array_list) # label array

# decompose sequences into 3-mers
X = df_val['seq']
X = SeqTo3mer(X)

# load tokenizer
with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# tokenize and zero padding
X_pad = TokenPad(X, 498, 'post', tokenizer)

# load model
biLSTMAtt = keras.models.load_model('./trained-model/')

# prediction
predictions = biLSTMAtt.predict(X_pad)
argmax_bilstmatt = np.argmax(predictions, axis=1)

# argmax true labels
argmax_valid = np.argmax(y_b, axis=1)

## Confusion matrix

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (classification_report,
							 confusion_matrix,
							 roc_auc_score)

label_list = ['5.8S-rRNA', '5S-rRNA', 'CD-box', 'HACA-box', 'Intron-gp-I', 
              'Intron-gp-II', 'Leader', 'Riboswitch', 'Ribozyme', 'Y-RNA',
              'Y-RNA-like', 'miRNA ', 'tRNA']


def plot_cm(labels, argmax_prediction, normalize, fmt, title):
    cm = confusion_matrix(labels, argmax_prediction, normalize=normalize)
    fig = plt.figure(figsize=(10, 8), dpi=350)
    sns.set(font_scale=1.15)
    sns.heatmap(cm, cmap='Reds', fmt=fmt, square=True, annot=True)
    plt.title(title, fontsize=22)
    plt.ylabel("True label", fontsize=20)
    plt.xlabel("Predicted label", fontsize=20)
    ax = fig.add_subplot(111)
    ax.set_xticklabels(label_list)
    ax.set_yticklabels(label_list)
    plt.yticks(rotation='horizontal', fontsize=18)
    plt.xticks(rotation='vertical', fontsize=18)

    

In [ ]:
report = classification_report(argmax_valid, argmax_bilstmatt)
print(report)
title = 'Validation set'
plot_cm(argmax_valid, argmax_bilstmatt, 'true','.2f', title)

## Metrics

In [ ]:
cnf_matrix = confusion_matrix(argmax_valid, argmax_bilstmatt)

FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print(TPR, 'recall')
# Specificity or true negative rate
TNR = TN/(TN+FP) 
print(TNR, 'specificity')
# Precision or positive predictive value
PPV = TP/(TP+FP)
print(PPV, 'precision')
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)
print(ACC, 'accuracy')
# F1-score
F1 = (2*TP)/((2*TP)+FP+FN)
print(F1, 'f1')
# Matthews correlation coefficient (MCC)
MCC = ((TP*TN)-(FP*FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
print(MCC, 'MCC')

In [ ]:
# overall accuracy
print((sum(TP)+sum(TN))/(sum(TP)+sum(TN)+sum(FP)+sum(FN)), 'accuracy')

# overall sensitivity
print(sum(TP)/(sum(TP)+sum(FN)), 'sensitivity')

# overall specificity
print(sum(TN)/(sum(TN)+sum(FP)), 'specificity')

# overall precision
print(sum(TP)/(sum(TP)+sum(FP)), 'precision')

# overall f1-score
print((2*(sum(TP)))/((2*(sum(TP)))+sum(FP)+sum(FN)), 'f1-score')

# ROC and Precision-recall curves

In [ ]:
Y_test = y_b
y_score = pred_bilstmatt

In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# references
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html

In [ ]:
# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(13):
    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                        y_score[:, i])
    average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
    y_score.ravel())
average_precision["micro"] = average_precision_score(Y_test, y_score,
                                                     average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'
      .format(average_precision["micro"]))

In [ ]:
plt.figure()
plt.step(recall['micro'], precision['micro'], where='post')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
    .format(average_precision["micro"]))

In [ ]:
# Precision-recall multiclass

colors = ['r', 'g', 'b', 'c', 'sienna', 'y', 'grey', 'darkorange', 'lime', 
                'purple', 'cyan', 'palegreen', 'magenta']

classes = ['5.8S-rRNA', '5S-rRNA', 'CD-box', 'HACA-box', 'Intron-gp-I', 'Intron-gp-II', 'Leader',
           'Riboswitch', 'Ribozyme', 'Y-RNA', 'Y-RNA-like', 'miRNA', 'tRNA']


n_classes = 13

plt.figure(figsize=(4,4), dpi=350)
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
pr_auc = []

for i, color in zip(range(n_classes), colors):
    l, = plt.plot(recall[i], precision[i], color=color, lw=2)
    lines.append(l)
    labels.append('{0} ({1:0.3f})'
                  ''.format(classes[i], average_precision[i]))
    pr_auc.append(average_precision[i])
    
plt.plot(recall['micro'], precision['micro'], color='black', linestyle='dashed', lw=1.5)
plt.xlim([-0.015, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontsize=20)
plt.ylabel('Precision', fontsize=20)
#plt.title('Precision-Recall curve')
#plt.legend(lines, labels, loc=(0, 1.5), prop=dict(size=14), ncol=2)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tick_params(axis='both', which='minor', labelsize=14)
plt.show()

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)

for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [ ]:
# Plot ROC curves
plt.figure(figsize=(4,4), dpi=350)

colors = cycle(['r', 'g', 'b', 'c', 'sienna', 'y', 'grey', 'darkorange', 'lime', 
                'purple', 'cyan', 'palegreen', 'magenta'])

label_list = ['5.8S-rRNA', '5S-rRNA', 'CD-box', 'HACA-box', 'Intron-gp-I', 'Intron-gp-II', 'Leader',
           'Riboswitch', 'Ribozyme', 'Y-RNA', 'Y-RNA-like', 'miRNA', 'tRNA']


roc_auc_list = []


for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color,
             label='{0} '
             ''.format(label_list[i]))
    roc_auc_list.append(roc_auc[i])


plt.plot(fpr['micro'], tpr['micro'], color='black', linestyle='dashed', lw=1, label='micro-average')
plt.plot([0, 1], [0, 1], 'k-',lw=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tick_params(axis='both', which='minor', labelsize=14)
plt.show()

# t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
def predict(model, X):
    
    """ Function used to extract the context vector and attention weights while predicting samples """
    
    # input: model, data, attention layer index, output distrbution from attention layer
    
    predictions = tf.keras.Model(inputs = model.input, 
                                 outputs = [model.output,
                                            model.layers[3].output[0],
                                            model.layers[3].output[1]])
    
    output, context, weights = predictions.predict(X)
    
    # return as dataframe
    new_df = pd.DataFrame()
    new_df['prediction'] = np.argmax(output, axis=1)
    new_df['c_vector'] = list(context)
    new_df['weights'] = weights
    
    return new_df

In [ ]:
def plot_tsne(df, p):
    
    " Used for plotting t-SNE results "

    colors = ['r', 'g', 'b', 'c', 'sienna', 'y', 'grey', 'darkorange', 'lime', 'purple', 'cyan', 'palegreen', 'magenta']

    classes = ['5.8S-rRNA', '5S-rRNA', 'CD-box', 'HACA-box', 'Intron-gp-I', 'Intron-gp-II', 'Leader',
               'Riboswitch', 'Ribozyme', 'Y-RNA', 'Y-RNA-like', 'miRNA', 'tRNA']
    
    plt.figure(figsize=(7, 6), dpi=350)
    
    for target, color in zip(classes, colors):
        indicesToKeep = output['true'] == target
    
        plt.scatter(df.loc[indicesToKeep, 'X']
                   , df.loc[indicesToKeep, 'Y']
                   , c = color
                   , s = 50
                   , alpha=0.9
                   , edgecolors='black'
                   , linewidths=0.6)
        
    plt.title('Perplexity: {}'.format(p), fontsize=14)
    plt.xlabel('X', fontsize=14)
    plt.ylabel('Y', fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.tick_params(axis='both', which='minor', labelsize=12)
    #plt.legend(classes)

In [ ]:
output = predict(biLSTMAtt, X_pad)
output['true'] = list(df_val['labels'])
output[list(range(64))] = pd.DataFrame(output.c_vector.tolist(), index= output.index)

In [ ]:
perplexities = [10, 25, 50]

for p in perplexities:

    X_embedded = TSNE(n_components=2, perplexity=p, init='pca', n_jobs=3).fit_transform(c)
    df_tsne = pd.DataFrame(data = X_embedded,
                           columns = ['X', 'Y'])
    df_tsne['prediction'] = output['prediction']

    plot_tsne(df_tsne, p)

## Attention weights

In [ ]:
# predict samples
prediction = predict(biLSTMAtt, X_pad)

In [ ]:
# store data as a dataframe
att_weights = pd.DataFrame()
att_weights[list(range(498))] = pd.DataFrame(prediction.weights.tolist(), index= prediction.index)
att_weights['prediction'] = prediction['prediction']
attention = att_weights.iloc[:, :-1]

In [ ]:
import seaborn as sns

fig = plt.figure(figsize=(20, 20), dpi=350)

yticks = [1, 24, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325 ]

yticklabels = ['', '5.8S-rRNA', '5S-rRNA', 'CD-box', 'HACA-box', 'Intron-gp-I', 'Intron-gp-II',
        'Leader', 'Riboswitch', 'Ribozyme', 'Y-RNA', 'Y-RNA-like', 'miRNA', 'tRNA'] 

ax1 = sns.heatmap(attention, square=True, cbar_kws={"shrink": 0.5})
ax1.set_yticks(yticks)
ax1.set_yticklabels(yticklabels, fontsize=20)
ax1.tick_params(axis='y', length=20)
plt.xlabel('Token position', fontsize=24)
plt.show()